In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import time
from imp import reload

In [3]:
%matplotlib notebook
plt.style.use('bmh')

In [4]:
data = pd.read_csv('Data.csv')

In [5]:
data.set_index('visitor_id', inplace = True)

In [ ]:
data.columns

In [6]:
data["visits_int_arr"] = data['visits'].apply(lambda row : np.array([int(x) for x in row.split(' ')]))

In [ ]:
data['visits_week'] = data['visits_int_arr'].apply(lambda row : np.where((row%7) == 0, row//7,row//7 + 1 ))

In [ ]:
# no. of visits per person
data['visit_count'] = data['visits_int_arr'].apply(lambda x : len(x))

In [ ]:
# first day and last day of visit 
data['first_day'] = data['visits_int_arr'].apply(lambda x : np.min(x))
data['last_day'] = data['visits_int_arr'].apply(lambda x : np.max(x))

In [ ]:
data.head()

In [ ]:
data.loc[data['last_day'] == 910, :].head()

#### EDA

#### 1. Distribution of days of visits

In [ ]:
plt.title('Freq distribution of number of days of visit')
plt.hist(data['visit_count'], bins = 50);

#### 2. How does count of people going to mall vary over days 
- Days 1 to 7 are most likely Sunday to Saturday, days 1 to 7 get below % traffic (out of 290,000)  
1      9.561379   
2      7.049310   
3      7.880345   
4      8.465517   
5      9.981379   
6     15.895517   
7     17.248966  
- **Means F1 score of final week will give weightage proprtional to these percentages**

In [ ]:
day_wise_cnt = pd.Series(data = [0]*910, index = np.arange(1,911,1))

In [ ]:
day_wise_cnt.tail()

In [ ]:
data.tail()

In [ ]:
def get_full_vector(row):
    ser = pd.Series(data = [0]*910, index = np.arange(1,911,1))
    ser[row] +=1
    #print('done')
    return ser

In [ ]:
data_dense = data.loc[:,'visits_int_arr'].apply(func = get_full_vector)

In [ ]:
by_day_count = data_dense.apply(np.sum, axis = 0)

In [ ]:
by_day_pct = (by_day_count/290000)*100

In [ ]:
plt.figure()
#plt.plot(by_day_pct.index.values, by_day_pct.values, 'bo-')
plt.plot(by_day_count[:28], 'bo-')
plt.title('Plot of total visitors in first four weeks - shows seasonality at weely level', fontsize = 'small')

In [ ]:
plt.figure()
#plt.plot(by_day_pct.index.values, by_day_pct.values, 'bo-')
plt.plot(by_day_pct[:28], 'bo-')
plt.title('Plot of total visitors in first four weeks - shows seasonality at weely level', fontsize = 'small')

In [ ]:
by_day_pct[0:28]

In [ ]:
#### plot with 6 month period
plt.figure()
plt.plot(by_day_pct[:184], 'bo-')
#plt.subplot(3,1,2)
#plt.plot(by_day_pct[365:730], 'bo-')
#plt.subplot(3,1,3)
#plt.plot(by_day_pct[730:], 'bo-')

In [ ]:
plt.figure()
plt.plot(by_day_pct[184:365], 'bo-')

In [ ]:
plt.figure()
plt.plot(by_day_pct[365:549], 'bo-')

In [ ]:
plt.figure()
plt.plot(by_day_pct[549:733], 'bo-')

In [ ]:
plt.figure()
plt.plot(by_day_pct[733:], 'bo-')

In [ ]:
data['visit_count'].unique()

#### weekly pattern os visits

In [ ]:
def get_full_weekly_matrix(row):
    ser = pd.Series(data = [0]*130, index = np.arange(1,131,1))
    ser[row] +=1
    #print('done')
    return ser

In [ ]:
data_dense_weekly = data.loc[:,'visits_week'].apply(func = get_full_weekly_matrix)


In [ ]:
by_week_count = data_dense_weekly.apply(np.sum, axis = 0)
by_week_pct = (by_week_count/290000)*100

In [ ]:
plt.figure()
#plt.plot(by_day_pct.index.values, by_day_pct.values, 'bo-')
plt.plot(by_week_count, 'bo-')
plt.title('Plot of total visitors', fontsize = 'small')

In [ ]:
plt.figure()
#plt.plot(by_day_pct.index.values, by_day_pct.values, 'bo-')
plt.plot(by_week_count[0:53], 'bo-')
plt.title('Plot of total visitors in first 52 weeks ', fontsize = 'small')

In [ ]:
plt.figure()
#plt.plot(by_day_pct.index.values, by_day_pct.values, 'bo-')
plt.plot(by_week_count[53:106], 'bo-')
plt.title('Plot of total visitors in next 52 weeks', fontsize = 'small')

In [ ]:
plt.figure()
#plt.plot(by_day_pct.index.values, by_day_pct.values, 'bo-')
plt.plot(by_week_count[106:], 'bo-')
plt.title('Plot of total visitors in last 25 weeks', fontsize = 'small')

#### time to first visit , and last visit from 910

In [ ]:
plt.figure()
plt.hist(data['first_day'], bins = 50);

In [ ]:
plt.figure()
plt.hist(data['last_day'], bins = 50);

#### average time between visits

In [ ]:
def avg_time_bw_visits(row):
    tbv_total = 0
    for i in range(len(row)-1):
        tbv_total += row[i+1] - row[i]
    return tbv_total/len(row)

In [ ]:
data['avg_tbv'] = data.loc[:,'visits_int_arr'].apply(avg_time_bw_visits)

In [ ]:
data.head()

In [ ]:
plt.figure()
plt.hist(data['avg_tbv'], bins = 100);
plt.title('Distribution of avg tbv', fontsize = 'small')

In [ ]:
data.head()

In [ ]:
plt.figure()
plt.scatter(data['visit_count'], data['avg_tbv'], alpha = 0.5, s = 10)
plt.xlabel('visit_count')
plt.ylabel('abg_tbv')

#### week level features 
- weekly frequency 
  - avg, cut by start week, going by x weeks
  - median
- periodicity in weeks
  - lag 1 to 5 averages
- recency 
  - weeks since last visit 
  - counts by visits in each of last x weeks

In [8]:
import data_prep as dp

In [ ]:
reload(dp)

In [ ]:
s = data.loc[0:5,'visits_week']
s

In [ ]:
kwargs ={
    'aw' : 24,
    'lb' :24
}
s.apply(dp.avg_wkly_visits, **kwargs)

In [ ]:
kwargs ={
    'aw' : 128,
    'lb' :128
}
s.apply(dp.median_wkly_visits, **kwargs)

In [ ]:
kwargs ={
    'aw' : 128,
    'lb' :128,
    'lag' : 10
}
s.apply(dp.avg_time_wks_bw_visits, **kwargs)

In [ ]:
kwargs ={
    'aw' : 128,
    'lb' :128,
    'lag' : 5
}
s.apply(dp.median_time_wks_bw_visits, **kwargs)

In [ ]:
kwargs ={
    'aw' : 66,
    'xth_week' :0
}
s.apply(dp.count_visits_last_x_week, **kwargs)

In [ ]:
s

In [ ]:
kwargs ={
    'aw' : 66
}
s.apply(dp.weeks_since_last_visit, **kwargs)

#### Day level features
- % visits on 1,2,3,4,5,6,7

** Fit a model and do error analysis

In [93]:
reload(dp)

<module 'data_prep' from '/Users/sumad/Documents/ShoppingMallDatachallenge/data_prep.py'>

In [40]:
kwargs ={
    'ad' : 100,
    'day' : 1
}
s.apply(dp.avg_visits_by_wkday, **kwargs)

visitor_id
1    0.00
2    0.00
3    0.00
4    0.03
5    0.00
Name: visits_int_arr, dtype: float64

In [56]:
kwargs ={
    'ad' : 896,
    'day' : 7,
    'lag' : 1
}
s.apply(dp.avg_time_days_bw_visits, **kwargs)

visitor_id
1    133.00
2    304.50
3     94.50
4    357.00
5    204.75
Name: visits_int_arr, dtype: float64

In [11]:
s = data.loc[0:5,'visits_int_arr']
s

visitor_id
1    [30, 84, 126, 135, 137, 179, 242, 342, 426, 45...
2    [24, 53, 75, 134, 158, 192, 194, 211, 213, 238...
3    [51, 143, 173, 257, 446, 491, 504, 510, 559, 6...
4    [1, 20, 22, 92, 124, 149, 211, 335, 387, 390, ...
5    [34, 51, 56, 106, 110, 121, 163, 233, 266, 275...
Name: visits_int_arr, dtype: object

In [78]:
kwargs ={
    'ad' : 560,
    'day' : 6,
    'xth_week' : 7
}
s.apply(dp.ind_visit_last_x_wkday, **kwargs)

visitor_id
1    0
2    0
3    1
4    0
5    0
Name: visits_int_arr, dtype: int64

In [89]:
kwargs ={
    'ad' : 126,
    'day' : 6
}
s.apply(dp.days_since_last_wday_visited, **kwargs)

visitor_id
1   -100
2   -100
3   -100
4    106
5     92
Name: visits_int_arr, dtype: int64

In [99]:
kwargs ={
    'target_week' : 4
}
s.apply(dp.create_mclass_target, **kwargs)

visitor_id
1    0
2    0
3    0
4    1
5    0
Name: visits_int_arr, dtype: int64